# Metadata Explorer

A notebook to explore the metadata so we know where to look for things!

### Import some libraries

In [1]:
import os
import sys

import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sns
import json

from itertools import cycle

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import permutation_importance

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error

from scipy.stats import linregress


# there is a FutureWarning in sklearn StandardScalar which is really annoying. This ignores it.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

try:
  import google.colab
  IN_COLAB = True
  !pip install adjustText
  from google.colab import drive
  drive.mount('/content/drive')
  datadir = '/content/drive/MyDrive/Projects/CF/Adelaide/CF_Data_Analysis'
except ImportError:
  IN_COLAB = False
  datadir = '..'

from adjustText import adjust_text

import cf_analysis_lib

### Read the data

In [2]:
sequence_type = 'MGI_minion'
metadata = cf_analysis_lib.read_metadata(datadir, sequence_type)
metadata.head(5)

,minion,MGI,pwCF_ID,Sample date,IP vs OP,Hospital,Room,Age,Age groups,Paediatric vs Adult,...,Sum of meds,Sum of antifungals,Sum of steroid + mabs,DNA_extraction_ conc,SAGC ULN,DNA Conc. (ng/ul),Index I7,Index I5,Mean_Size_BP,Total Clusters Passing Filter (Million)
NAME,,,,,,,,,,,,,,,,,,,,,
634207_20180510_S,NaN,634207_20180510_S,634207,5/10/2018,IP,WCH,Adol Rm9,17,3,Paediatric,...,1,0,0,0.000,SAGCFN_22_01856,7.82,CGGACGATTC,CCACCACCTA,651,2.9
634207_20180517_S,NaN,634207_20180517_S,634207,5/17/2018,IP,WCH,Adol Rm9,17,3,Paediatric,...,1,0,0,0.134,SAGCFN_22_01827,22.80,AGCGATAG,CCTATCCT,633,2.4
715927_20180205_S,715927_20180205_S,715927_20180205_S,715927,2/05/2018,OP,WCH,Level 6 DK Office,13,3,Paediatric,...,1,0,0,0.326,SAGCFN_22_01797,16.50,TAATGCGC,AGGCGAAG,516,3.4
715927_20180213_S,NaN,715927_20180213_S,715927,2/13/2018,IP,WCH,Adol Room 11,13,3,Paediatric,...,3,0,0,0.234,SAGCFN_22_01811,31.00,TCCGCGAA,CCTATCCT,443,2.7
715927_20180226_S,NaN,715927_20180226_S,715927,2/26/2018,OP,WCH,OPD 8,13,3,Paediatric,...,2,0,0,0.108,SAGCFN_22_01833,15.10,TAACTTGGTC,GATTCACGAC,510,2.6


In [3]:
sorted(list(metadata.columns))

['1 Cephalexin_PO',
 '1 Flucloaxcillin_PO',
 '1 Itraconazole (Lozenoc)_PO',
 '1 Sulfamethoxazole_trimethoprim (Bactrim)_PO',
 '2 Amikacin_INH',
 '2 Amoxicillin & Potassium clavulanate (Aug Duo)_PO',
 '2 Amphotericin B (Ambisome)_INH',
 '2 Azithromycin_PO',
 '2 Ceftazidime_INH',
 '2 Ciprofloxacin_PO',
 '2 Clarithromycin_PO',
 '2 Clofazimine PO',
 '2 Colistin_IHN',
 '2 prednisolone_PO',
 '2 tobramycin_INH',
 '3 Azithromycin_IV',
 '3 Aztreonam_IV',
 '3 Cefopime_IV',
 '3 Ceftazidime_IV',
 '3 Imipenem',
 '3 Ivacaftor (Kalydeco)',
 '3 Meropenem_IV',
 '3 Methylpredinosolone_IV',
 '3 Omalizumab_SC',
 '3 piperacillin sodium, tazobactam sodium (Tazocin)_IV',
 '3 tobramycin_IV',
 '4 Amikacin_IV',
 '4 Cefoxitin_IV',
 '4 Colistin_IV',
 'Age',
 'Age groups',
 'Antibiotics (duration)',
 'Antibiotics_YN',
 'Best FEV1',
 'CF gene 1',
 'CF gene 2',
 'CFLD',
 'CH4/H2 ratio_corrected',
 'CH4_Corrected',
 'CH4_Uncorrected',
 'CO2',
 'CS_Achromobacter xylosoxidans',
 'CS_Acremonium species',
 'CS_Aspergillu

In [5]:
antibiotics = ['1 Cephalexin_PO', '1 Flucloaxcillin_PO', '1 Itraconazole (Lozenoc)_PO', '1 Sulfamethoxazole_trimethoprim (Bactrim)_PO', '2 Amikacin_INH', '2 Amoxicillin & Potassium clavulanate (Aug Duo)_PO', '2 Amphotericin B (Ambisome)_INH', '2 Azithromycin_PO', '2 Ceftazidime_INH', '2 Ciprofloxacin_PO', '2 Clarithromycin_PO', '2 Clofazimine PO', '2 Colistin_IHN', '2 prednisolone_PO', '2 tobramycin_INH', '3 Azithromycin_IV', '3 Aztreonam_IV', '3 Cefopime_IV', '3 Ceftazidime_IV', '3 Imipenem', '3 Ivacaftor (Kalydeco)', '3 Meropenem_IV', '3 Methylpredinosolone_IV', '3 Omalizumab_SC', '3 piperacillin sodium, tazobactam sodium (Tazocin)_IV', '3 tobramycin_IV', '4 Amikacin_IV', '4 Cefoxitin_IV', '4 Colistin_IV']

In [6]:
antibiotics

['1 Cephalexin_PO',
 '1 Flucloaxcillin_PO',
 '1 Itraconazole (Lozenoc)_PO',
 '1 Sulfamethoxazole_trimethoprim (Bactrim)_PO',
 '2 Amikacin_INH',
 '2 Amoxicillin & Potassium clavulanate (Aug Duo)_PO',
 '2 Amphotericin B (Ambisome)_INH',
 '2 Azithromycin_PO',
 '2 Ceftazidime_INH',
 '2 Ciprofloxacin_PO',
 '2 Clarithromycin_PO',
 '2 Clofazimine PO',
 '2 Colistin_IHN',
 '2 prednisolone_PO',
 '2 tobramycin_INH',
 '3 Azithromycin_IV',
 '3 Aztreonam_IV',
 '3 Cefopime_IV',
 '3 Ceftazidime_IV',
 '3 Imipenem',
 '3 Ivacaftor (Kalydeco)',
 '3 Meropenem_IV',
 '3 Methylpredinosolone_IV',
 '3 Omalizumab_SC',
 '3 piperacillin sodium, tazobactam sodium (Tazocin)_IV',
 '3 tobramycin_IV',
 '4 Amikacin_IV',
 '4 Cefoxitin_IV',
 '4 Colistin_IV']

In [7]:
combinations = {
'Amikacin' : ['2 Amikacin_INH', '4 Amikacin_IV'],
'Azithromycin' : ['2 Azithromycin_PO', '3 Azithromycin_IV'],
'Tobramycin' : ['2 tobramycin_INH', '3 tobramycin_IV'],
'Colistin' : ['2 Colistin_IHN', '4 Colistin_IV'],
'Ceftazidime' : ['2 Ceftazidime_INH', '3 Ceftazidime_IV'],
'Prednisolone' : ['2 prednisolone_PO', '3 Methylpredinosolone_IV']
}

In [12]:
metadata[(metadata['2 Amikacin_INH'] == 1) | (metadata['4 Amikacin_IV'] == 1)]

,minion,MGI,pwCF_ID,Sample date,IP vs OP,Hospital,Room,Age,Age groups,Paediatric vs Adult,...,Sum of meds,Sum of antifungals,Sum of steroid + mabs,DNA_extraction_ conc,SAGC ULN,DNA Conc. (ng/ul),Index I7,Index I5,Mean_Size_BP,Total Clusters Passing Filter (Million)
NAME,,,,,,,,,,,,,,,,,,,,,
770560_20190109_S,NaN,770560_20190109_S,770560,1/09/2019,IP,WCH,Adolescent 4,12,3,Paediatric,...,4,0,0,0.0710,SAGCFN_22_01842,12.00,CTTGCTATTC,CGTGACTCTA,395,4.2
770560_20181218_S,NaN,770560_20181218_S,770560,12/18/2018,IP,WCH,Adolescent 4,12,3,Paediatric,...,3,0,0,0.1610,SAGCFN_22_01821,14.90,TCTCGCGC,AGGCGAAG,630,3.2
670829_20171001_S,NaN,670829_20171001_S,670829,10/01/2017,IP,WCH,Adolescent 6,15,3,Paediatric,...,1,0,0,0.2800,SAGCFN_22_01803,41.80,CGGCTATG,CCTATCCT,375,2.2
670829_20171005_S,NaN,670829_20171005_S,670829,10/05/2017,IP,WCH,Adolescent 6,15,3,Paediatric,...,1,0,0,0.1020,SAGCFN_22_01836,5.16,ATGTTGCCAC,ATGGCTTCTA,668,4.0
650003_20180207_S,650003_20180207_S,650003_20180207_S,650003,2/07/2018,IP,WCH,Adol Room 1,17,3,Paediatric,...,5,0,0,0.5520,SAGCFN_22_01778,8.28,GAATTCGT,ATAGAGGC,476,5.7
650003_20180213_S,NaN,650003_20180213_S,650003,2/13/2018,IP,WCH,Level 6 Treatment,17,3,Paediatric,...,5,0,0,0.2890,SAGCFN_22_01800,27.60,TAATGCGC,GTACTGAC,572,3.5
670829_20180201_S,NaN,670829_20180201_S,670829,2/01/2018,IP,WCH,Day Surgery 17,16,3,Paediatric,...,2,1,0,0.0810,SAGCFN_22_01840,8.58,AGGTAACAAC,AGCATCGTGC,469,3.1
698917_20180119_S,698917_20180119_S,698917_20180119_S,698917,1/19/2018,IP,WCH,Adol Room 1,14,3,Paediatric,...,3,0,0,1.7600,SAGCFN_22_01747,23.20,TCCGGAGA,CCTATCCT,383,6.7
698917_20180128_S,698917_20180128_S,698917_20180128_S,698917,1/28/2018,IP,WCH,Adol Room 1,14,3,Paediatric,...,3,0,0,0.3370,SAGCFN_22_01793,21.00,TAATGCGC,TATAGCCT,495,2.2


In [13]:
tmpdf = pd.DataFrame()
for new_col, col_array in combinations.items():
    tmpdf[new_col] = metadata[col_array].any(axis=1).astype(int)
tmpdf

,Amikacin,Azithromycin,Tobramycin,Colistin,Ceftazidime,Prednisolone
NAME,,,,,,
634207_20180510_S,0,0,1,0,0,0
634207_20180517_S,0,0,1,0,0,0
715927_20180205_S,0,0,0,0,0,0
715927_20180213_S,0,0,1,0,0,0
715927_20180226_S,0,0,1,0,0,0
...,...,...,...,...,...,...
1651490_20180206_S,0,0,1,0,0,0
1651490_20171215_S,0,0,1,0,0,0
1658447_20171006_S,0,0,0,0,1,0


In [14]:
tmpdf.sum()

Amikacin        10
Azithromycin     8
Tobramycin      47
Colistin         4
Ceftazidime     22
Prednisolone    10
dtype: int64